In [108]:
# %pip install beautifulsoup4 requests pymysql
# %pip install recipe-scrapers

In [1]:
from bs4 import BeautifulSoup
import requests
from recipe_scrapers import scrape_me, WebsiteNotImplementedError
import time
import pymysql
import pickle
import json
delay = 1

In [5]:
urls_dict = {
    'Browse dinner collections:': 'https://www.acouplecooks.com/category/recipes/everyday-healthy-dinner-recipes/',
    'Browse fast dinner collections:': 'https://www.acouplecooks.com/category/recipes/fast-dinner-ideas/',
    'Browse top dinner recipes:': 'https://www.acouplecooks.com/category/recipes/special-occasion-dinner-recipes/',
    'Browse side dish & salad collections:': 'https://www.acouplecooks.com/category/recipes/side-dishes-salad-recipes/',
    'Browse dessert collections:': 'https://www.acouplecooks.com/category/recipes/dessert-recipes/',
    'Browse brunch and breakfast recipes:':'https://www.acouplecooks.com/category/recipes/breakfast-brunch-recipes/',
    'Browse essentials & basics:':'https://www.acouplecooks.com/category/recipes/essential-recipes/',
    'Browse appetizer and snack collections:': 'https://www.acouplecooks.com/category/recipes/appetizers-healthy-snack-recipes/',
    'Browse top collections:': 'https://www.acouplecooks.com/category/recipe-ideas/'
}

In [20]:
def get_http_links(kw, url):
    host = 'https://www.acouplecooks.com'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    h2 = soup.find('h2', text=kw)
    links = h2.find_next_sibling().find_all('a', class_='quick-link')
    https_links = set()
    for link in links:
        if link['href'].startswith('https'):
            https_links.add(link['href'])
        else:
            https_links.add(host+link['href'])
    return https_links

In [37]:
def get_recipe(https_link):
    r = requests.get(https_link)
    html_content = r.text
    soup = BeautifulSoup(html_content, 'lxml')
    ls = {button.get('data-mv-create-link-href') for button in soup.find_all('button', class_='mv-list-link mv-to-btn')}
    return ls

get_recipe('https://www.acouplecooks.com/best-green-salad-recipes/')

{'https://www.acouplecooks.com/best-tossed-salad-recipe/',
 'https://www.acouplecooks.com/blackberry-salad-with-balsamic-vinaigrette/',
 'https://www.acouplecooks.com/caesar-salad/',
 'https://www.acouplecooks.com/chopped-salad-with-radishes-and-blue-cheese/',
 'https://www.acouplecooks.com/easy-arugula-salad/',
 'https://www.acouplecooks.com/easy-mexican-salad/',
 'https://www.acouplecooks.com/fennel-orange-salad/',
 'https://www.acouplecooks.com/incredible-chopped-salad/',
 'https://www.acouplecooks.com/italian-salad/',
 'https://www.acouplecooks.com/kale-salad-recipe/',
 'https://www.acouplecooks.com/spinach-pear-and-apple-pecan-salad/',
 'https://www.acouplecooks.com/spinach-salad/',
 'https://www.acouplecooks.com/strawberry-spinach-salad/'}

In [38]:
def get_all_links(urls_dict):
    all_links = set()
    for kw, url in urls_dict.items():
        https_links = get_http_links(kw, url)
        for https_link in https_links:
            all_links.update(get_recipe(https_link))
    return all_links

In [44]:
all_links = get_all_links(urls_dict)
all_links.remove('')

20
44
57
74
88
121
137
142
154
165
176
176
189
208
223
257
264
274
278
280
284
292
295
300
305
324
339
340
341
343
351
351
352
361
364
364
364
364
364
364
364
364
364
364
364
364
375
387
415
445
452
461
466
477
487
507
522
538
540
541
546
556
567
574
579
598
608
622
644
649
650
655
660
670
675
678
700
713
715
725
725
743
757
770
777
781
792
797
799
799
802
806
808
808
808
808
808
808
808
808
808
808
816
816
816
831
831
853
867
873
878
894
903
903
914
918
926
926
931
933
934
934
937
939
961
961
983
999
999
1028
1036
1039
1039
1039


In [45]:
len(all_links)

1038

In [48]:
all_content = dict()
for i,link in enumerate(list(all_links)):
    try:
        scraper = scrape_me(link)
        all_content[i] = scraper.to_json()
        all_content[i]["recipe_id"] = i + 1
        print(f'retrieved {i} recipes, {i/len(all_links)*100:.2f}%')
        time.sleep(delay) #be polite, delay some time after making a request
    except WebsiteNotImplementedError:
        print(f"Website {link} not supported")

# serialize the data and save it to a file
with open('data.pickle', 'wb') as f:
    pickle.dump(all_content, f)

Website https://www.wellplated.com/spicy-shrimp-guacamole-bites/ not supported
retrieved 1 recipes, 0.10
retrieved 2 recipes, 0.19
retrieved 3 recipes, 0.29
retrieved 4 recipes, 0.39
retrieved 5 recipes, 0.48
Website https://jessicainthekitchen.com/cinnamon-french-toast-recipe/ not supported
retrieved 7 recipes, 0.67
retrieved 8 recipes, 0.77
retrieved 9 recipes, 0.87
retrieved 10 recipes, 0.96
retrieved 11 recipes, 1.06
retrieved 12 recipes, 1.16
retrieved 13 recipes, 1.25
retrieved 14 recipes, 1.35
retrieved 15 recipes, 1.45
retrieved 16 recipes, 1.54
retrieved 17 recipes, 1.64
retrieved 18 recipes, 1.73
retrieved 19 recipes, 1.83
retrieved 20 recipes, 1.93
retrieved 21 recipes, 2.02
retrieved 22 recipes, 2.12
retrieved 23 recipes, 2.22
retrieved 24 recipes, 2.31
retrieved 25 recipes, 2.41
Website https://www.africanbites.com/apple-cobbler/ not supported
retrieved 27 recipes, 2.60
retrieved 28 recipes, 2.70
retrieved 29 recipes, 2.79
retrieved 30 recipes, 2.89
retrieved 31 recipes, 2

In [100]:
all_content[1]

{'author': 'Sonja Overhiser',
 'canonical_url': 'https://www.acouplecooks.com/greek-fries/',
 'category': 'Side dish',
 'host': 'acouplecooks.com',
 'image': 'https://www.acouplecooks.com/wp-content/uploads/2020/10/Greek-Fries-013-225x225.jpg',
 'ingredients': ['6 medium Yukon gold potatoes (1 1/2 to 1 3/4 pounds)*',
  '2 tablespoons olive oil',
  '1 tablespoon homemade Greek Seasoning',
  '1/2 teaspoon kosher salt',
  '1/4 cup feta crumbles',
  '2 tablespoons finely chopped curly parsley',
  '2 tablespoons finely chopped red onion',
  '4 lemon wedges, for serving',
  'Optional: tzatziki, lemon dill sauce (our favorite), or aioli, for dipping'],
 'instructions': 'Preheat the oven to 425 degrees Fahrenheit. Evenly space the oven racks to ensure even cooking.\nMeanwhile, cut the potatoes into fries\n(Watch this video or use a fry cutter.) Wash the potatoes, leaving the skins on. Slice off the ends of each potato, then slice off part of the side to make a base. Place the potato half cut s

In [93]:
with open('data.pickle', 'rb') as f:
    all_content = pickle.load(f)

In [106]:
all_content[1]['instructions_list']

['Preheat the oven to 425 degrees Fahrenheit. Evenly space the oven racks to ensure even cooking.',
 'Meanwhile, cut the potatoes into fries',
 '(Watch this video or use a fry cutter.) Wash the potatoes, leaving the skins on. Slice off the ends of each potato, then slice off part of the side to make a base. Place the potato half cut side down and cut off a 3/8-inch slice, then lay the slice on its side and cut it into several long strips. Repeat until all potatoes have been cut into fries. The pieces can be uneven, but aim for as uniform of thickness as possible. See the video for full instructions!',
 'Soak in cold water (10 minutes)',
 'Place the cut fries into a medium bowl and rinse with cold water until water is clear. Then fill the bowl with cold water and allow to rest for 10 minutes (while the oven preheats). Then drain the potatoes and dry them thoroughly with a clean dish towel.',
 'Season the fries',
 'Add the fries to a dry bowl and toss them with the olive oil, Greek seaso

In [107]:
def make_kb(n, fn):
    kb = {}
    for i in range(n):
        try:
            if all_content[i+1]['category']:
                object_type = all_content[i+1]['category'].lower()
            recipe = {
                    'id': all_content[i+1]['recipe_id'],
                    'ingredients': all_content[i+1]['ingredients'],
                    'instructions': all_content[i+1]['instructions'],
                    'instructions_list': all_content[i+1]['instructions_list'],
                    'ratings': all_content[i+1]['ratings'],
                    'title': all_content[i+1]['title'],
                    'total_time': all_content[i+1]['total_time'],
                    'yields': all_content[i+1]['yields'],
                }
            if object_type not in kb:
                kb[object_type] = [recipe]
            else:
                kb[object_type].append(recipe)
        except KeyError:
            continue
    with open(fn, 'w') as fp:
        json.dump(kb, fp, indent=6)

In [110]:
make_kb(6, 'smallKnowledgeBase.json') #simple knowledgebase

In [109]:
make_kb(len(all_content), 'knowledgeBase.json')